In [33]:
import pandas as pd
import sys

sys.path.insert(0,'../satori')
import scipy.stats as stats
from scipy.stats import hypergeom
from postprocess import *

In [17]:
root_dir = "/s/chromatin/p/nobackup/Saira/original/satori/"
'''baseline'''
attn_attr_path = ("results/arabidopsis/NEW_SPLIT/baseline/E1/" , "Interactions_attnattr_abs_logits/unique_interactions_ATTNATTR.csv")
raw_attn_path = ("results/arabidopsis/NEW_SPLIT/baseline/E1/" , "Interactions_SATORI/Arabidopsis_interactions_SATORI.csv")
dfim_path = ("results/arabidopsis/NEW_SPLIT/baseline/E1/", "Interactions_FIS_orig/unique_interactions_FIS.csv")
'''baseline entropy'''
attn_attr_path_entropy = ("results/arabidopsis/NEW_SPLIT/baseline/Entropy/New_EE_noseed/" , "Interactions_attnattr_abs_logits/Arabidopsis_interactions_ATTNATTR.csv")
raw_attn_path_entropy = ("results/arabidopsis/NEW_SPLIT/baseline/Entropy/New_EE_noseed/", "Interactions_SATORI/Arabidopsis_interactions_SATORI.csv")
'''deeper'''
raw_attr_path_deeper = ("results/arabidopsis/NEW_SPLIT/sei/E_no_residual_noseed_run2/" , "Interactions_SATORI/Arabidopsis_interactions_SATORI.csv")
attn_attr_path_deeper = ("results/arabidopsis/NEW_SPLIT/sei/E_no_residual_noseed_run2/" , "Interactions_attnattr_abs_logits/Arabidopsis_interactions_ATTNATTR.csv")

'''deeper entropy'''
raw_attr_path_deeper_entropy = ("results/arabidopsis/NEW_SPLIT/sei/E1_entropy/" , "unique_interactions_SATORI.csv")
attn_attr_path_deeper_entropy = ("results/arabidopsis/NEW_SPLIT/sei/E1_entropy/" , "Interactions_attnattr_abs_logits/Arabidopsis_interactions_AttnAttr.csv")

In [19]:
output_paths = [dfim_path, raw_attn_path, attn_attr_path, raw_attn_path_entropy, attn_attr_path_entropy , raw_attr_path_deeper, attn_attr_path_deeper, raw_attr_path_deeper_entropy, attn_attr_path_deeper_entropy]

In [20]:
df_intrs = pd.read_csv("/s/chromatin/p/nobackup/Saira/motif_databases/Arabidopsis/arath_interactions.csv", delimiter=",")
tfs = set()
for i, row in df_intrs.iterrows():
    tf1, tf2 = row[0], row[1]
    if pd.notna(tf1) and pd.notna(tf2):  # safer than np.isnan for mixed types
        tfs.add((tf1.upper(), tf2.upper()))
        tfs.add((tf2.upper(), tf1.upper()))

/tmp/ipykernel_1335892/2302450906.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tf1, tf2 = row[0], row[1]


In [23]:
def read_motifs_by_modelss(exp_dir, data_name ,  df_annotate=None):
    # Load tomtom 
    tomt = pd.read_csv(exp_dir + "/Motif_Analysis/tomtom/tomtom.tsv", delimiter="\t")#deep_ent_tomtom.tsv
    filters = {}
    for i, row in tomt.iterrows():
        filters.setdefault(row["Query_ID"],[]).append((row["q-value"],row["Target_ID"]))

    annotations = set()
    for key in filters.keys():
        temp = filters[key]
        annotations.add(temp[0][1])

    cleaned_set = {x for x in annotations if not (isinstance(x, float) and math.isnan(x))}
    unique_tfs = {}
    if df_annotate is not None and data_name == "human_promoters":
        for motif in cleaned_set:
            unique_tfs[motif] = get_annotation(motif, annotation_data=df_annotate, single_TF=False)
    else:
        for motif in cleaned_set:
            unique_tfs[motif] = motif.split('_')[1].strip('.tnt').upper()
        
    return unique_tfs


In [24]:
res_ppi = {"DFIM" : [], "Attn-Raw" : [], "Attn-Attr" : [],
           "Attn-Raw + Entropy" : [], "Attn-Attr + Entropy" : [],
           "Attn-Raw [Deep]": [], "Attn-Attr [Deep]": [],
           "Attn-Raw + Entropy [Deep]": [], "Attn-Attr + Entropy [Deep]": []}

for i in range(len(output_paths)):
    print(root_dir , output_paths[i])
    unique_tfs = read_motifs_by_modelss(root_dir + output_paths[i][0], "Arabidopsis")
    n, matching_df  = match_with_ppi(root_dir + output_paths[i][0] + output_paths[i][1] , tfs, unique_tfs) 
    k = matching_df.shape[0]
    all_combinations = create_all_combinations(unique_tfs)

    # Print the number of combinations
    print(f"Total number of combinations: {len(all_combinations)}")
    count = 0
    for comb in all_combinations:
        if len(comb) == 1:
            continue
        if (comb[0],comb[1]) in tfs or (comb[1],comb[0]) in tfs:
            count += 1
    N = len(all_combinations)
    K = count
    print(N,K,n,k)
    # Calculate p-value
    p_value = stats.hypergeom.sf(k-1, N, K, n)  
    res_ppi[list(res_ppi.keys())[i]].extend([len(unique_tfs),N,K,n,k,p_value])
    print(f"P-value: {p_value}")


/s/chromatin/p/nobackup/Saira/original/satori/ ('results/arabidopsis/NEW_SPLIT/baseline/E1/', 'Interactions_FIS_orig/unique_interactions_FIS.csv')
Total number of combinations: 703
703 2 586 1
P-value: 0.9724988146040777
/s/chromatin/p/nobackup/Saira/original/satori/ ('results/arabidopsis/NEW_SPLIT/baseline/E1/', 'Interactions_SATORI/Arabidopsis_interactions_SATORI.csv')
Total number of combinations: 703
703 2 132 0
P-value: 1.0
/s/chromatin/p/nobackup/Saira/original/satori/ ('results/arabidopsis/NEW_SPLIT/baseline/E1/', 'Interactions_attnattr_abs_logits/unique_interactions_ATTNATTR.csv')
Total number of combinations: 703
703 2 538 2
P-value: 0.585415374889059
/s/chromatin/p/nobackup/Saira/original/satori/ ('results/arabidopsis/NEW_SPLIT/baseline/Entropy/New_EE_noseed/', 'Interactions_SATORI/Arabidopsis_interactions_SATORI.csv')
Total number of combinations: 630
630 2 469 2
P-value: 0.5538950715421302
/s/chromatin/p/nobackup/Saira/original/satori/ ('results/arabidopsis/NEW_SPLIT/baseli

In [28]:
all_combinations

{('ANAC042', 'AT1G12630'),
 ('ANAC042', 'ERF1'),
 ('ANAC055', 'ANAC042'),
 ('ANAC055', 'AT1G12630'),
 ('ANAC055', 'AT1G18960'),
 ('ANAC055', 'AT1G72740'),
 ('ANAC055', 'AT2G20400'),
 ('ANAC055', 'AT2G33550'),
 ('ANAC055', 'AT2G40260'),
 ('ANAC055', 'AT4G28140'),
 ('ANAC055', 'AT4G38000'),
 ('ANAC055', 'AT5G66940'),
 ('ANAC055', 'ATAF1'),
 ('ANAC055', 'ATHB40'),
 ('ANAC055', 'ATIDD11'),
 ('ANAC055', 'ATY19'),
 ('ANAC055', 'BHLH34'),
 ('ANAC055', 'BIM1'),
 ('ANAC055', 'BIM2'),
 ('ANAC055', 'BZR1'),
 ('ANAC055', 'COG1'),
 ('ANAC055', 'DEAR5'),
 ('ANAC055', 'DEL1'),
 ('ANAC055', 'ERF1'),
 ('ANAC055', 'ERF10'),
 ('ANAC055', 'ERF105'),
 ('ANAC055', 'ERF7'),
 ('ANAC055', 'GBF3'),
 ('ANAC055', 'MYB39'),
 ('ANAC055', 'MYB51'),
 ('ANAC055', 'MYB67'),
 ('ANAC055', 'MYB92'),
 ('ANAC055', 'NAM'),
 ('ANAC055', 'OBP1'),
 ('ANAC055', 'OBP3'),
 ('ANAC055', 'TCP20'),
 ('ANAC055', 'WRKY31'),
 ('ANAC055', 'WRKY6'),
 ('ANAC055', 'WRKY7'),
 ('AT1G18960', 'ANAC042'),
 ('AT1G18960', 'AT1G12630'),
 ('AT1G18960

In [31]:
res_ppi_df = pd.DataFrame(res_ppi).T
res_ppi_df.columns = ["# of unique Tfs", "N(all possible interactions)", "K(exist in database)", "n(identified by model)", "k(matches with database)", "p-value"]

In [32]:
res_ppi_df.to_csv(root_dir + "arabidopsis_ppi_analysis.csv")
res_ppi_df

,# of unique Tfs,N(all possible interactions),K(exist in database),n(identified by model),k(matches with database),p-value
DFIM,39.0,703.0,2.0,586.0,1.0,0.972499
Attn-Raw,39.0,703.0,2.0,132.0,0.0,1.000000
Attn-Attr,39.0,703.0,2.0,538.0,2.0,0.585415
Attn-Raw + Entropy,36.0,630.0,2.0,469.0,2.0,0.553895
Attn-Attr + Entropy,36.0,630.0,2.0,511.0,2.0,0.657658
Attn-Raw [Deep],32.0,496.0,2.0,140.0,2.0,0.079260
Attn-Attr [Deep],32.0,496.0,2.0,163.0,2.0,0.107551
Attn-Raw + Entropy [Deep],39.0,741.0,3.0,441.0,3.0,0.210214
Attn-Attr + Entropy [Deep],39.0,741.0,3.0,64.0,0.0,1.000000
